Gathering data from various sources such as Excel files, databases, APIs, and third-party services is a common requirement in data processing and analysis. Below is a guide on how to extract data from these different sources using Python.

1. Extracting Data from Excel Files
You can use the pandas library to read Excel files.

In [2]:
import pandas as pd

def extract_from_excel(file_path, sheet_name=0):
    """
    Extracts data from an Excel file.
    :param file_path: Path to the Excel file.
    :param sheet_name: Name or index of the sheet to read.
    :return: DataFrame containing the data.
    """
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    print(f"Data extracted from Excel: {file_path}")
    return data


2. Extracting Data from Databases
For relational databases (like MySQL, PostgreSQL, SQLite), you can use the sqlite3 module or SQLAlchemy.

Using SQLite

In [5]:
import sqlite3

def extract_from_database(db_name, query):
    """
    Extracts data from a SQLite database.
    :param db_name: Name of the database file.
    :param query: SQL query to execute.
    :return: DataFrame containing the data.
    """
    conn = sqlite3.connect(db_name)
    data = pd.read_sql_query(query, conn)
    conn.close()
    print(f"Data extracted from database: {db_name}")
    return data


3. Extracting Data from APIs
You can use the requests library to interact with REST APIs.

In [8]:
import requests

def extract_from_api(url, params=None):
    """
    Extracts data from a REST API.
    :param url: URL of the API endpoint.
    :param params: Dictionary of query parameters (optional).
    :return: JSON response from the API.
    """
    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an error for bad responses
    data = response.json()
    print(f"Data extracted from API: {url}")
    return data


4. Extracting Data from Third-Party Services
Third-party services often provide their APIs to fetch data. The process is similar to extracting data from standard APIs. For example, fetching data from a service like Salesforce, Twitter, or any other platform would usually involve using their respective SDKs or REST APIs.

In [11]:
# Example for a hypothetical third-party service
def extract_from_third_party_service(api_endpoint, headers):
    """
    Extracts data from a third-party service.
    :param api_endpoint: Endpoint of the third-party API.
    :param headers: Authorization or other necessary headers.
    :return: JSON response from the third-party service.
    """
    response = requests.get(api_endpoint, headers=headers)
    response.raise_for_status()  # Raise an error for bad responses
    data = response.json()
    print(f"Data extracted from third-party service: {api_endpoint}")
    return data


Complete ETL Example
You can combine all these extraction methods into a complete ETL process. Here’s how you can structure the ETL pipeline:

Summary
This code provides a flexible ETL pipeline that can gather data from various sources, including Excel files, databases, APIs, and third-party services. You can customize the function parameters to fit your specific needs and data structures.

In [40]:
import pandas as pd
import sqlite3
import requests
import os

def extract_from_excel(file_path, sheet_name=0):
    """Extracts data from an Excel file."""
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    print(f"Data extracted from Excel: {file_path}")
    return data

def extract_from_database(db_name, query):
    """Extracts data from a SQLite database."""
    conn = sqlite3.connect(db_name)
    data = pd.read_sql_query(query, conn)
    conn.close()
    print(f"Data extracted from database: {db_name}")
    return data

def extract_from_api(api_url, params, headers=None):
    """Extracts data from a REST API."""
    response = requests.get(api_url, params=params, headers=headers)
    response.raise_for_status()  # Raise an error for bad responses
    data = pd.json_normalize(response.json())
    print(f"Data extracted from API: {api_url}")
    return data

def extract_from_third_party(third_party_url, headers=None):
    """Extracts data from a third-party service."""
    response = requests.get(third_party_url, headers=headers)
    response.raise_for_status()
    data = pd.json_normalize(response.json())
    print(f"Data extracted from third-party service: {third_party_url}")
    return data

def transform_data(data):
    """Transforms the extracted data (cleaning and formatting)."""
    # Example transformation: removing leading/trailing whitespace
    data_cleaned = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    # Further transformation steps can be added here
    print("Data transformation completed.")
    return data_cleaned

def load_to_database(data, db_name, table_name):
    """Loads the cleaned data into a SQLite database."""
    conn = sqlite3.connect(db_name)
    data.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    print(f"Data loaded into database: {db_name}, table: {table_name}")

def etl_pipeline(excel_file, db_name, query, api_url, params, third_party_url, headers):
    """Full ETL pipeline that gathers data from various sources."""
    
    # Extract from Excel
    excel_data = extract_from_excel(excel_file)
    
    # Extract from Database
    db_data = extract_from_database(db_name, query)
    
    # Extract from API
    api_data = extract_from_api(api_url, params, headers)
    
    # Extract from Third-party Service
    third_party_data = extract_from_third_party(third_party_url, headers)
    
    # Combine all extracted data into one DataFrame
    combined_data = pd.concat([excel_data, db_data, api_data, third_party_data], ignore_index=True)
    
    # Transform
    cleaned_data = transform_data(combined_data)
    
    # Load the cleaned data to the database
    load_to_database(cleaned_data, db_name, 'cleaned_data')
    
    return cleaned_data

# Example Usage
if __name__ == "__main__":
    # Define paths and parameters
    excel_file_path = 'C:/Users/USER/Downloads/data.xlsx'  # Ensure this file exists
    db_name = 'datawarehouse.db'  # SQLite database name
    sql_query = 'SELECT * FROM your_table;'  # Ensure this table exists in the database
    api_url = 'https://api.example.com/data'  # Replace with actual API URL
    params = {'param1': 'value1'}  # Adjust parameters as needed
    third_party_url = 'https://thirdparty.example.com/data'  # Replace with actual URL
    headers = {'Authorization': 'Bearer YOUR_TOKEN'}  # Optional headers
    
    # Check if the Excel file exists
    if not os.path.exists(excel_file_path):
        raise FileNotFoundError(f"The Excel file at {excel_file_path} was not found.")
    
    # Run the ETL pipeline
    try:
        final_data = etl_pipeline(excel_file_path, db_name, sql_query, api_url, params, third_party_url, headers)
        print(final_data)
    except Exception as e:
        print(f"An error occurred: {e}")


Data extracted from Excel: C:/Users/USER/Downloads/data.xlsx
An error occurred: Execution failed on sql 'SELECT * FROM your_table;': no such table: your_table
